In [1]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [3]:
#data analysis
import pandas as pd 
import numpy as np 

#visualising 
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import matplotlib.patches as patches
import shap

#ml
import lightgbm as lgb
import lightgbm as LGBMRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, AdaBoostClassifier 
from sklearn.datasets import make_regression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier

from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("diagnosis-of-covid-19-and-its-clinical-spectrum.csv")
# Drop the null columns where all values are null
# df = df.dropna(axis='columns', how='all')
# Drop the null rows
# df = df.dropna()
df.columns = [x.lower().strip().replace(' ','_') for x in df.columns]

In [5]:
#print('Size of the data', df.shape)

In [6]:
#df.columns.values

In [7]:
#df.head()

In [8]:
#transform target variable into numeric 

In [9]:
#df['sars_cov_2_exam_result'] = df['sars_cov_2_exam_result'].replace(['negative','positive'], [0,1])
#sns.countplot(df['sars_cov_2_exam_result'])

In [10]:
#print("There are {}% target values with 1".format(100 *df['sars_cov_2_exam_result'].value_counts()[1]/df.shape[0]))

In [11]:
## Examine missing values
# def missing_data(data):
#     total = data.isnull().sum()
#     percent = (data.isnull().sum()/data.isnull().count()*100)
#     tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
#     types = []
#     for col in data.columns:
#         dtype = str(data[col].dtype)
#         types.append(dtype)
#     tt['Types'] = types
#     return(np.transpose(tt))

In [12]:
#missing_data(df)

In [13]:
## Handling missing values 
#!pip install quilt
#import missingno as msno
#The msno.matrix nullity matrix is a data-dense display which lets you quickly visually analyse data completion.
#msno.matrix(df.head(20000))

In [14]:
#The missingno correlation heatmap measures nullity correlation: 
#how strongly the presence or absence of one variable affects the presence of another:

#a = msno.heatmap(df, sort='ascending')
#a


In [15]:
#The dendrogram allows you to more fully correlate variable completion, 
#revealing trends deeper than the pairwise ones visible in the correlation heatmap:

#a2 = msno.dendrogram(df)
#a2

In [16]:
# Number of each type of column
#df.dtypes.value_counts()

In [17]:
# Number of unique classes in each object column
df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

patient_id                                            5644
sars_cov_2_exam_result                                   2
patient_addmited_to_regular_ward_1_yes_0_no              2
patient_addmited_to_semi_intensive_unit_1_yes_0_no       2
patient_addmited_to_intensive_care_unit_1_yes_0_no       2
respiratory_syncytial_virus                              2
influenza_a                                              2
influenza_b                                              2
parainfluenza_1                                          2
coronavirusnl63                                          2
rhinovirus_enterovirus                                   2
coronavirus_hku1                                         2
parainfluenza_3                                          2
chlamydophila_pneumoniae                                 2
adenovirus                                               2
parainfluenza_4                                          2
coronavirus229e                                         

In [18]:
#Now that we have dealt with the categorical variables and the outliers, let's continue with the EDA. One way to try and understand the data is by looking for correlations between the features and the target. We can calculate the Pearson correlation coefficient between every variable and the target using the .corr dataframe method.

#The correlation coefficient is not the greatest method to represent "relevance" of a feature, but it does give us an idea of possible relationships within the data. Some general interpretations of the absolute value of the correlation coefficent are:

#00-.19 “very weak”
#.20-.39 “weak”
#.40-.59 “moderate”
#.60-.79 “strong”
#.80-1.0 “very strong”

In [19]:
##Correlations
# Find correlations with the target and sort
#correlations = df.corr()['sars_cov_2_exam_result'].sort_values()

# Display correlations
#print('Most Positive Correlations:\n', correlations.tail(15))
#print('\nMost Negative Correlations:\n', correlations.head(15))


In [20]:
#first ten correlations between the features in dataset
features = df.columns.values[2:112]
corrs_ = df[features].corr().abs().unstack().sort_values(kind="quicksort").reset_index()
corrs_ = corrs_[corrs_['level_0'] != corrs_['level_1']]
#corrs_.head(10)

In [21]:
#corrs_.tail(10)

In [22]:
#The correlation between the features is small

In [23]:
#Correlations clustermap
# corrs = df.corr()
# plt.figure(figsize = (20, 8))
# # Heatmap of correlations
# sns.heatmap(corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = False, vmax = 0.8)
# plt.title('Clustermap');

In [24]:
#make some visuals
# def plot_dist_col(column):
#     pos__df = df[df['sars_cov_2_exam_result'] ==1]
#     neg__df = df[df['sars_cov_2_exam_result'] ==0]

#     '''plot dist curves for train and test weather data for the given column name'''
#     fig, ax = plt.subplots(figsize=(10, 10))
#     sns.distplot(pos__df[column].dropna(), color='green', ax=ax).set_title(column, fontsize=16)
#     sns.distplot(neg__df[column].dropna(), color='purple', ax=ax).set_title(column, fontsize=16)
#     plt.xlabel(column, fontsize=15)
#     plt.legend(['Positive sars cov 2 exam result', 'Negative sars cov 2 exam result'])
#     plt.show()
# plot_dist_col('patient_age_quantile')

In [25]:
# fig = px.pie( values=df.groupby(['sars_cov_2_exam_result']).size().values,names=df.groupby(['sars_cov_2_exam_result']).size().index)
# fig.update_layout(
#     font=dict(
#         size=15,
#         color="black" ) )   
# py.iplot(fig)



In [26]:
#Encoding variables: find a way to encode (represent) these variables as numbers before handing them off to the model

#fill in mean for floats
for c in df.columns:
    if df[c].dtype=='float16' or  df[c].dtype=='float32' or  df[c].dtype=='float64':
        df[c].fillna(df[c].mean())

#fill in -999 for categoricals
df = df.fillna(-999)
# Label Encoding
for f in df.columns:
    if df[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df[f].values))
        df[f] = lbl.transform(list(df[f].values))
        
#print('Labelling done.')  

In [27]:
##feature selection 
## Find the optimal feature subset using an evaluation measure. 
#The choice of evaluation metric distinguish the three main strategies 
#of feature selection algorithms: the wrapper strategy, the filter 
#strategy, and the embedded strategy

#Filter methods:

#information gain
#chi-square test
#correlation coefficient
#variance threshold

#Wrapper methods:
#recursive feature elimination
#sequential feature selection algorithms

#Embedded methods:
#L1 (LASSO) regularization
#decision tree

In [28]:
#In our case, we remove some useless, redundant variables. 
#We will use three methods for feature selection: 
#Remove collinear features, remove features with greater than a 
#threshold percentage of missing values, keep only the most relevant 
#features using feature importances from a model

In [29]:
#remove collinear varialbe: variables that highly correlated to one another 
#can decrease model's availability to learn, decrease model interpretability and 
#decrease generalization performance on the test set

# Threshold for removing correlated variables
threshold = 0.92

# Absolute value correlation matrix
corr_matrix = df.corr().abs()
corr_matrix.head()
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
#upper.head()

In [30]:
# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

# print('There are %d columns to remove.' % (len(to_drop)))
dataset = df.drop(columns = to_drop)
# print('Data shape: ', dataset.shape)
# print('Size of the data', dataset.shape)

In [31]:
#to_drop

In [32]:
#remove missing values
#dataset missing values (in percent)
dataset_missing = (dataset.isnull().sum() / len(dataset)).sort_values(ascending = False)
#dataset_missing.head()

In [33]:
#identify missing values above threshold
dataset_missing_ = dataset_missing.index[dataset_missing > 0.85]

all_missing = list(set(dataset_missing_))
#print('There are %d columns with more than 85%% missing values' % len(all_missing))
dataset = dataset.drop(columns = all_missing)
#print('Data shape: ', dataset.shape)

In [34]:
#feature selection through feature importance
cat_features = [i for i in dataset.columns if str(dataset[i].dtype) in ['object', 'category']]

if len(cat_features) > 0:
    dataset[cat_features] = dataset[cat_features].astype('category')


df_lgb = dataset.copy()
for i in cat_features:
    df_lgb[i] = dataset[i].cat.codes

df_lgb.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df_lgb.columns]

dataset_labels = df_lgb['sars_cov_2_exam_result']
df_lgb_ = df_lgb.copy()
df_lgb = df_lgb.drop(['patient_id', 
                      'sars_cov_2_exam_result', 
                      'patient_addmited_to_regular_ward_1_yes_0_no',
                      'patient_addmited_to_semi_intensive_unit_1_yes_0_no',
                      'patient_addmited_to_intensive_care_unit_1_yes_0_no'
                ], axis=1)
x = df_lgb.copy()

In [35]:
# Initialize an empty array to hold feature importances
feature_importances = np.zeros(df_lgb.shape[1])

# Create the model with several hyperparameters
model = lgb.LGBMClassifier(objective='binary', boosting_type = 'goss', n_estimators = 5000, class_weight = 'balanced')

# Fit the model twice to avoid overfitting
for i in range(2):
    # Split into training and validation set
    dataset_features, valid_features, dataset_features_y, valid_y = train_test_split(x, dataset_labels, test_size = 0.20, random_state = i)
    
    # Train using early stopping
    model.fit(dataset_features, dataset_features_y, early_stopping_rounds=100, eval_set = [(valid_features, valid_y)], 
              eval_metric = 'auc', verbose = 200)
    
    # Record the feature importances
    feature_importances += model.feature_importances_

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[25]	valid_0's auc: 0.649972	valid_0's binary_logloss: 0.610572
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10]	valid_0's auc: 0.603197	valid_0's binary_logloss: 0.643876


In [43]:
#average feature importances! 
feature_importances = feature_importances / 2
feature_importances = pd.DataFrame({'feature': list(df_lgb.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)
feature_importances.head()

,feature,importance
0,patient_age_quantile,276.5
5,neutrophils,45.5
7,proteina_c_reativa_mg_dl,33.5
1,hematocrit,29.0
3,respiratory_syncytial_virus,26.5


In [56]:
# Find the features with zero importance
zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
#print('There are %d features with 0.0 importance' % len(zero_features))
#feature_importances.tail()

In [57]:
def plot_feature_importances(df, threshold = 0.9):

    plt.rcParams['font.size'] = 18
    
    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()
    df['cumulative_importance'] = np.cumsum(df['importance_normalized'])

#     # Make a horizontal bar chart of feature importances
#     plt.figure(figsize = (10, 6))
#     ax = plt.subplot()
    
#     # Need to reverse the index to plot most important on top
#     ax.barh(list(reversed(list(df.index[:15]))), 
#             df['importance_normalized'].head(15), 
#             align = 'center', edgecolor = 'k')
    
#     # Set the yticks and labels
#     ax.set_yticks(list(reversed(list(df.index[:15]))))
#     ax.set_yticklabels(df['feature'].head(15))
    
#     # Plot labeling
#     plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
#     plt.show()
    
#     # Cumulative importance plot
#     plt.figure(figsize = (8, 6))
#     plt.plot(list(range(len(df))), df['cumulative_importance'], 'r-')
#     plt.xlabel('Number of Features'); plt.ylabel('Cumulative Importance'); 
#     plt.title('Cumulative Feature Importance');
#     plt.show();
    
    importance_index = np.min(np.where(df['cumulative_importance'] > threshold))
#     print('%d features required for %0.2f of cumulative importance' % (importance_index + 1, threshold))
    
    return df

In [58]:
#df.head()

In [59]:
norm_feature_importances = plot_feature_importances(feature_importances)

In [60]:
#remove the features that have zero importance.
df_lgb = df_lgb.drop(columns = zero_features)
#print('Dataset shape: ', df_lgb.shape)

In [61]:
#re-run the model to see if it identifies any more features with zero importance, (kind of like manual recursive feature eilmination)

def identify_zero_importance_features(train, train_labels, iterations = 2):
    
    #Initialize an empty array to hold feature importances
    feature_importances = np.zeros(train.shape[1])

    #Create the model with several hyperparameters
    model = lgb.LGBMClassifier(objective='binary', boosting_type = 'goss', n_estimators = 10000, class_weight = 'balanced')
    
    #Fit the model multiple times to avoid overfitting
    for i in range(iterations):

        #Split into training and validation set
        train_features, valid_features, train_y, valid_y = train_test_split(train, train_labels, test_size = 0.25, random_state = i)

        # Train using early stopping
        model.fit(train_features, train_y, early_stopping_rounds=100, eval_set = [(valid_features, valid_y)], 
                  eval_metric = 'auc', verbose = 200)

        # Record the feature importances
        feature_importances += model.feature_importances_ / iterations
    
    feature_importances = pd.DataFrame({'feature': list(train.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)
    
    # Find the features with zero importance
    zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
    print('\nThere are %d features with 0.0 importance' % len(zero_features))
    
    return zero_features, feature_importances

In [62]:
#zero_features

In [63]:
second_round_zero_features, feature_importances = identify_zero_importance_features(df_lgb, dataset_labels)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[21]	valid_0's auc: 0.644206	valid_0's binary_logloss: 0.616384
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.627722	valid_0's binary_logloss: 0.633327
Early stopping, best iteration is:
[108]	valid_0's auc: 0.640551	valid_0's binary_logloss: 0.632409

There are 0 features with 0.0 importance


In [64]:
#norm_feature_importances = plot_feature_importances(feature_importances, threshold = 0.95)

In [65]:
# # Threshold for cumulative importance
threshold = 0.95

# Extract the features to keep
features_to_keep = list(norm_feature_importances[norm_feature_importances['cumulative_importance'] < threshold]['feature'])
features_to_keep.append('patient_addmited_to_intensive_care_unit_1_yes_0_no')
features_to_keep.append('patient_addmited_to_regular_ward_1_yes_0_no')
features_to_keep.append('patient_addmited_to_semi_intensive_unit_1_yes_0_no')
features_to_keep.append('patient_id')
features_to_keep.append('sars_cov_2_exam_result')

        


# Create new datasets with smaller features
dataset_small = df_lgb_[features_to_keep]

#dataset_small.head()
    

In [211]:

# shap_values = shap.TreeExplainer(model).shap_values(valid_features)
# shap.summary_plot(shap_values, valid_features)

In [212]:
#summarize the effects of top features 
# shap.summary_plot(shap_values[0], valid_features, max_display=30)    

In [213]:
train_df = dataset_small
features = list(train_df)
features.remove('patient_id')
features.remove('sars_cov_2_exam_result')
features.remove('patient_addmited_to_intensive_care_unit_1_yes_0_no')
features.remove('patient_addmited_to_regular_ward_1_yes_0_no')
features.remove('patient_addmited_to_semi_intensive_unit_1_yes_0_no')
target = 'sars_cov_2_exam_result'

In [214]:
# Split Train and Validation
X_train = train_df.drop('patient_id',axis=1)
X_train = X_train.drop(['sars_cov_2_exam_result',
                        'patient_addmited_to_regular_ward_1_yes_0_no',
                        'patient_addmited_to_semi_intensive_unit_1_yes_0_no',
                        'patient_addmited_to_intensive_care_unit_1_yes_0_no'],
                       axis=1)
target = train_df['sars_cov_2_exam_result']

X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  target,
                                                  test_size=0.30, 
                                                  random_state=2020, 
                                                  stratify=target)

In [215]:
#model selection

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    ]
for classifier in classifiers:
    model = classifier.fit(X_train, y_train)
    print(classifier)
    print("model score: %.3f" % model.score(X_val, y_val))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')
model score: 0.874
SVC(C=0.025, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)
model score: 0.901
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
model score: 0.888
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
     

In [216]:
#looking at Random Forest Classifier alone 
rf = RandomForestClassifier(n_estimators=2000) 
rf = rf.fit(X_train, y_train) 
acc = rf.score(X_val, y_val) 
print(f"Training Data Score: {rf.score(X_train, y_train)}") 
print(f"Testing Data Score: {rf.score(X_val, y_val)}")

Training Data Score: 0.9167088607594936
Testing Data Score: 0.8990554899645808


In [56]:
#Hyperparameter optimization aka find the best set of parameters for the algorithm 
#create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [10,20,30],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
#                           cv = 3, n_jobs = -1, verbose = 2)
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                             n_jobs = -1, verbose = 2)


In [57]:
# Train the model with GridSearch
grid_search.fit(X_train, y_train)
grid_search.best_params_
best_grid = grid_search.best_estimator_
print(f"Training Data Score: {best_grid.score(X_train, y_train)}")
print(f"Testing Data Score: {best_grid.score(X_val, y_val)}")

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 16.7min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed: 18.1min finished


Training Data Score: 0.9045569620253164
Testing Data Score: 0.9002361275088547


In [58]:
from sklearn.metrics import classification_report

In [59]:
#use the best model to predict labels on the test set and print classification report 
# y_pred_best = best_model_rf.predict(X_train)
# print(classification_report(y_train, y_pred_best))

NameError: name 'best_model_rf' is not defined

# Save the Model

In [66]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
# import joblib
# filename = 'best_model_rf.sav'
# joblib.dump(best_model_rf, filename)
# filename1 = 'best_model_gnb.sav'
# joblib.dump(best_model_gnb, filename1)

In [70]:
#for dash app figure 
fig_feature_importances = feature_importances[:18]
fig_feature_importances

,feature,importance
0,patient_age_quantile,1238.0
3,respiratory_syncytial_virus,130.0
8,influenza_b_rapid_test,92.0
4,neutrophils,64.5
1,hematocrit,61.5
6,proteina_c_reativa_mg_dl,58.5
5,urea,38.0
13,strepto_a,26.5
7,potassium,17.5
2,serum_glucose,15.5


In [77]:
fig_feature_importances['log10_value'] = np.log10(fig_feature_importances.loc[:,'importance']) 
log_features = fig_feature_importances
log_features

,feature,importance,log10_value
0,patient_age_quantile,1238.0,3.092721
3,respiratory_syncytial_virus,130.0,2.113943
8,influenza_b_rapid_test,92.0,1.963788
4,neutrophils,64.5,1.809560
1,hematocrit,61.5,1.788875
6,proteina_c_reativa_mg_dl,58.5,1.767156
5,urea,38.0,1.579784
13,strepto_a,26.5,1.423246
7,potassium,17.5,1.243038
2,serum_glucose,15.5,1.190332


In [38]:
import plotly.graph_objects as go

fig = go.Figure(go.Bar(
            x=log_features["log10_value"],
            y=log_features["feature"],
            orientation='h'))
fig.update_layout(yaxis=dict(autorange="reversed"))

fig.show()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices